In [10]:
import pandas as pd
import cv2
import os
from keras.layers import Dense, Flatten
from keras.models import Model
from tqdm import tqdm
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.utils import to_categorical
from keras.applications.resnet50 import ResNet50
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
# print('CPU:', tf.config.list_physical_devices(device_type='CPU'))
# print('GPU:', tf.config.list_physical_devices(device_type='GPU'))

In [2]:
file_path = './sfddd/driver_imgs_list.csv'
df = pd.read_csv(file_path)
df.head()

,subject,classname,img
0,p002,c0,img_44733.jpg
1,p002,c0,img_72999.jpg
2,p002,c0,img_25094.jpg
3,p002,c0,img_69092.jpg
4,p002,c0,img_92629.jpg


In [7]:
IMAGE_SIZE = (224,224)
# num: how many images in sequence do you want to load
def load_train_data(num):
    images = []
    labels = []
    dataset = './sfddd/imgs/train'
    for folder in os.listdir(dataset):
        label = folder
        subdir = f'{dataset}/{folder}'
        for file in tqdm(os.listdir(subdir)[:num]):
            img_path = os.path.join(subdir, file)
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, IMAGE_SIZE) # to match ResNet
            images.append(image)
            labels.append(int(label[-1]))
    images = np.array(images)
    labels = np.array(labels)
    return [images, labels]


In [11]:
X, y = load_train_data(100)
y = to_categorical(y)
print(len(X), len(y))
print(X.shape, y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

100%|██████████| 100/100 [00:00<00:00, 316.88it/s]


1000 1000
(1000, 224, 224, 3) (1000, 10)


In [12]:
# ResNet
model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
for layer in model.layers:
    layer.trainable = False
x = Flatten()(model.output)
x = Dense(1024, activation='relu')(x)
x = Dense(1024, activation='relu')(x)
outputs = Dense(10, activation='softmax')(x)
model = Model(inputs=model.input, outputs=outputs)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=100, validation_split=0.3)

Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 28s 4s/step - accuracy: 0.0764 - loss: 146.0005 - val_accuracy: 0.0900 - val_loss: 251.2826
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 15s 3s/step - accuracy: 0.1857 - loss: 196.1806 - val_accuracy: 0.2654 - val_loss: 56.1180
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 15s 3s/step - accuracy: 0.2491 - loss: 52.8528 - val_accuracy: 0.2701 - val_loss: 25.1765
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 15s 3s/step - accuracy: 0.4235 - loss: 16.1276 - val_accuracy: 0.2844 - val_loss: 9.3138
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 15s 3s/step - accuracy: 0.5050 - loss: 4.0995 - val_accuracy: 0.5024 - val_loss: 4.5791
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 15s 3s/step - accuracy: 0.7197 - loss: 1.9465 - val_accuracy: 0.6303 - val_loss: 2.6717
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 15s 3s/step - accuracy: 0.8709 - loss: 0.6108 - val_accuracy: 0.6730 - val_loss: 1.6873
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 14s 3s/step - accuracy: 0.9507 - loss: 0.1610 - val_accuracy: 0.7346 - val_loss: 1.157

In [13]:
model.evaluate(X_test, y_test)

10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 579ms/step - accuracy: 0.8138 - loss: 0.8855


[1.0393413305282593, 0.79666668176651]